In order to ensure that there is a sufficient number of data sets in cross -verification, we choose to use 80%of the samples as a training set and 20%of the samples as tests. The data we use comes from api.openweathermap.org and passed through data cleaning, but for vacant values, due to the increase in extreme weather such as global warming, we adopt different filling methods for different models (in detail in the model part) Essence Our goal is the temperature of the temperature [minimum value, maximum, maximum] five days of prediction value.

1. Historical mean model: Simple, easy to implement time sequence prediction method. As a benchmark model to evaluate the performance of other complex models, its predictive effect will decrease significantly when the data has dynamic changes (such as trends or cyclical).

In [38]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_predictions(actual, predicted, num_features=1):
    mse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)  # R² calculation
    n = len(actual)  # Number of data points
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - num_features - 1)  # Adjusted R²
    return mse, mae, r2, adj_r2

# Define Historical Mean prediction function
def historical_mean_forecast(series, steps=5, start_date=None):
    mean_value = series.mean()
    forecast_values = [mean_value] * steps
    future_dates = [start_date + pd.Timedelta(days=i) for i in range(1, steps + 1)]
    forecast_df = pd.DataFrame({'DATE': future_dates, 'FORECAST': forecast_values})
    return forecast_df


directory = '.'  
output_file = 'output/AllCities_HistoricalMean.txt'  
os.makedirs('output', exist_ok=True)  

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("Historical Mean Results for All Cities:\n\n")

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        city_name = os.path.splitext(filename)[0]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)

        df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
        df = df[df['DATE'] <= '2024-11-19']  # Filter data before or on 2024-11-19
        df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2  
        df['TMAX'] = df['TMAX'].interpolate(method='linear')
        df['TMIN'] = df['TMIN'].interpolate(method='linear')
        df['TAVG'] = df['TAVG'].interpolate(method='linear')

        # Train-Test Split
        train_tmax = df['TMAX'][:-5]
        test_tmax = df['TMAX'][-5:]
        train_tmin = df['TMIN'][:-5]
        test_tmin = df['TMIN'][-5:]
        train_tavg = df['TAVG'][:-5]
        test_tavg = df['TAVG'][-5:]

        start_date = df['DATE'].max()  # Start date for predictions
        tmax_historical_forecast_df = historical_mean_forecast(train_tmax, steps=5, start_date=start_date)
        tmin_historical_forecast_df = historical_mean_forecast(train_tmin, steps=5, start_date=start_date)
        tavg_historical_forecast_df = historical_mean_forecast(train_tavg, steps=5, start_date=start_date)

        tmax_mse, tmax_mae, tmax_r2, tmax_adj_r2 = evaluate_predictions(test_tmax, tmax_historical_forecast_df['FORECAST'])
        tmin_mse, tmin_mae, tmin_r2, tmin_adj_r2 = evaluate_predictions(test_tmin, tmin_historical_forecast_df['FORECAST'])
        tavg_mse, tavg_mae, tavg_r2, tavg_adj_r2 = evaluate_predictions(test_tavg, tavg_historical_forecast_df['FORECAST'])

        combined_forecast_df = pd.DataFrame({
            'DATE': tmax_historical_forecast_df['DATE'],
            'TMIN': tmin_historical_forecast_df['FORECAST'],
            'TMAX': tmax_historical_forecast_df['FORECAST'],
            'TAVG': tavg_historical_forecast_df['FORECAST']
        })

        output_text = f"City: {city_name}\n"
        output_text += f"TMAX - MSE: {tmax_mse:.2f}, MAE: {tmax_mae:.2f}, R²: {tmax_r2:.4f}, Adjusted R²: {tmax_adj_r2:.4f}\n"
        output_text += f"TMIN - MSE: {tmin_mse:.2f}, MAE: {tmin_mae:.2f}, R²: {tmin_r2:.4f}, Adjusted R²: {tmin_adj_r2:.4f}\n"
        output_text += f"TAVG - MSE: {tavg_mse:.2f}, MAE: {tavg_mae:.2f}, R²: {tavg_r2:.4f}, Adjusted R²: {tavg_adj_r2:.4f}\n\n"
        output_text += "Future 5-Day Predictions (Combined):\n"
        output_text += combined_forecast_df.to_string(index=False)
        output_text += "\n\n" + "-" * 50 + "\n\n"

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(output_text)

        print(f"Processed {city_name}, results appended to {output_file}")


Processed KBNA, results appended to output/AllCities_HistoricalMean.txt
Processed KBOI, results appended to output/AllCities_HistoricalMean.txt
Processed KDCA, results appended to output/AllCities_HistoricalMean.txt
Processed KDEN, results appended to output/AllCities_HistoricalMean.txt
Processed KDTW, results appended to output/AllCities_HistoricalMean.txt
Processed KIAH, results appended to output/AllCities_HistoricalMean.txt
Processed KJFK, results appended to output/AllCities_HistoricalMean.txt
Processed KMIA, results appended to output/AllCities_HistoricalMean.txt
Processed KMSP, results appended to output/AllCities_HistoricalMean.txt
Processed KOKC, results appended to output/AllCities_HistoricalMean.txt
Processed KORD, results appended to output/AllCities_HistoricalMean.txt
Processed KPDX, results appended to output/AllCities_HistoricalMean.txt
Processed KPHX, results appended to output/AllCities_HistoricalMean.txt
Processed KPWM, results appended to output/AllCities_HistoricalM

2.OLS model: For the return of OLS to join the lagging project, we consider using the average value instead of FFILL or BFILL is to avoid the effect of extreme changes.

In [39]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore")

def evaluate_model(y_test, y_pred, X_test):
    n = len(y_test)  # Number of observations
    p = X_test.shape[1]  # Number of predictors
    r2 = r2_score(y_test, y_pred)  # R² score
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)  # Adjusted R²
    mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
    return r2, adj_r2, mse, rmse, mae

def generate_future_predictions(data, model_min, model_max, model_avg, days=5, start_date=None, lag_features=[]):
    future_predictions = []
    current_data = data.iloc[-1][lag_features].values.reshape(1, -1)
    current_date = pd.to_datetime(start_date)

    for day in range(1, days + 1):
        pred_min = model_min.predict(current_data)[0]
        pred_max = model_max.predict(current_data)[0]
        pred_avg = model_avg.predict(current_data)[0]

        future_predictions.append({
            'DATE': current_date + pd.Timedelta(days=day),
            'TMIN': pred_min,
            'TMAX': pred_max,
            'TAVG': pred_avg
        })

        current_data = np.roll(current_data, -3)  
        current_data[0, :3] = [pred_max, pred_min, pred_avg]  # Insert new predictions

    return pd.DataFrame(future_predictions)


directory = '.'  
output_file = 'output/AllCities_LinearRegression.txt'  
os.makedirs('output', exist_ok=True)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("Linear Regression Model Results for All Cities:\n\n")

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        city_name = os.path.splitext(filename)[0]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)

        df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')  # Handle invalid dates
        df = df[df['DATE'].notna()]  
        df = df[df['DATE'] <= '2024-11-19']  # Filter data before or on 2024-11-19
        df['TMAX'] = df['TMAX'].interpolate(method='linear')
        df['TMIN'] = df['TMIN'].interpolate(method='linear')
        df['PRCP'] = df['PRCP'].interpolate(method='linear')
        df['SNOW'] = df['SNOW'].interpolate(method='linear')
        df['SNWD'] = df['SNWD'].interpolate(method='linear')
        df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2

        for i in range(1, 10):
            df[f'TMAX_lag{i}'] = df['TMAX'].shift(i)
            df[f'TMIN_lag{i}'] = df['TMIN'].shift(i)
            df[f'TAVG_lag{i}'] = df['TAVG'].shift(i)
        df.dropna(inplace=True)

        lag_features = [f'TMAX_lag{i}' for i in range(1, 10)] + \
                       [f'TMIN_lag{i}' for i in range(1, 10)] + \
                       [f'TAVG_lag{i}' for i in range(1, 10)] + ['PRCP', 'SNOW', 'SNWD']

        X = df[lag_features]
        y_min = df['TMIN']
        y_max = df['TMAX']
        y_avg = df['TAVG']

        # Train-Test Split
        X_train, X_test, y_min_train, y_min_test, y_max_train, y_max_test, y_avg_train, y_avg_test = train_test_split(
            X, y_min, y_max, y_avg, test_size=0.2, random_state=42
        )

        model_min = LinearRegression()
        model_max = LinearRegression()
        model_avg = LinearRegression()

        model_min.fit(X_train, y_min_train)
        model_max.fit(X_train, y_max_train)
        model_avg.fit(X_train, y_avg_train)

        y_min_pred = model_min.predict(X_test)
        y_max_pred = model_max.predict(X_test)
        y_avg_pred = model_avg.predict(X_test)

        r2_min, adj_r2_min, mse_min, rmse_min, mae_min = evaluate_model(y_min_test, y_min_pred, X_test)
        r2_max, adj_r2_max, mse_max, rmse_max, mae_max = evaluate_model(y_max_test, y_max_pred, X_test)
        r2_avg, adj_r2_avg, mse_avg, rmse_avg, mae_avg = evaluate_model(y_avg_test, y_avg_pred, X_test)

        future_predictions = generate_future_predictions(df, model_min, model_max, model_avg, days=5, start_date='2024-11-19', lag_features=lag_features)

        output_text = f"City: {city_name}\n"
        output_text += f"TMIN Evaluation: R²: {r2_min:.4f}, Adjusted R²: {adj_r2_min:.4f}, MSE: {mse_min:.4f}, RMSE: {rmse_min:.4f}, MAE: {mae_min:.4f}\n"
        output_text += f"TMAX Evaluation: R²: {r2_max:.4f}, Adjusted R²: {adj_r2_max:.4f}, MSE: {mse_max:.4f}, RMSE: {rmse_max:.4f}, MAE: {mae_max:.4f}\n"
        output_text += f"TAVG Evaluation: R²: {r2_avg:.4f}, Adjusted R²: {adj_r2_avg:.4f}, MSE: {mse_avg:.4f}, RMSE: {rmse_avg:.4f}, MAE: {mae_avg:.4f}\n\n"
        output_text += "Future 5-Day Predictions (Combined):\n"
        output_text += future_predictions.to_string(index=False)
        output_text += "\n\n" + "-" * 50 + "\n\n"

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(output_text)

        print(f"Processed {city_name}, results appended to {output_file}")


Processed KBNA, results appended to output/AllCities_LinearRegression.txt
Processed KBOI, results appended to output/AllCities_LinearRegression.txt
Processed KDCA, results appended to output/AllCities_LinearRegression.txt
Processed KDEN, results appended to output/AllCities_LinearRegression.txt
Processed KDTW, results appended to output/AllCities_LinearRegression.txt
Processed KIAH, results appended to output/AllCities_LinearRegression.txt
Processed KJFK, results appended to output/AllCities_LinearRegression.txt
Processed KMIA, results appended to output/AllCities_LinearRegression.txt
Processed KMSP, results appended to output/AllCities_LinearRegression.txt
Processed KOKC, results appended to output/AllCities_LinearRegression.txt
Processed KORD, results appended to output/AllCities_LinearRegression.txt
Processed KPDX, results appended to output/AllCities_LinearRegression.txt
Processed KPHX, results appended to output/AllCities_LinearRegression.txt
Processed KPWM, results appended to ou

3. Random Forest: Random Forest Return is suitable for time sequence prediction. It can handle complex non -linear relationships, and there are less distribution of data, but we still need to deal with missing values. We choose linear filling.
In order to ensure that the computing time complexity does not exceed requirements, we simplify the random forest and control the number of decision -making trees and the maximum depth.

In [32]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore")

def evaluate_model(y_test, y_pred, X_test):
    n = len(y_test)  # Number of observations
    p = X_test.shape[1]  # Number of predictors
    r2 = r2_score(y_test, y_pred)  # R² score
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)  # Adjusted R²
    mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
    return r2, adj_r2, mse, rmse, mae

def generate_future_predictions(data, model_min, model_max, model_avg, days=5, start_date=None, lag_features=[]):
    future_predictions = []
    current_data = data.iloc[-1][lag_features].values.reshape(1, -1)
    current_date = pd.to_datetime(start_date)

    for day in range(1, days + 1):
        pred_min = model_min.predict(current_data)[0]
        pred_max = model_max.predict(current_data)[0]
        pred_avg = model_avg.predict(current_data)[0]

        future_predictions.append({
            'DATE': current_date + pd.Timedelta(days=day),
            'TMIN': pred_min,
            'TMAX': pred_max,
            'TAVG': pred_avg
        })

        current_data = np.roll(current_data, -3)  
        current_data[0, :3] = [pred_max, pred_min, pred_avg]  # Insert new predictions

    return pd.DataFrame(future_predictions)

directory = '.' 
output_file = 'output/AllCities_RandomForest.txt' 
os.makedirs('output', exist_ok=True) 

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("Random Forest Regressor Results for All Cities:\n\n")

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        city_name = os.path.splitext(filename)[0]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)

        df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')  # Handle invalid dates
        df = df[df['DATE'].notna()]  
        df = df[df['DATE'] <= '2024-11-19']  # Filter data before or on 2024-11-19

        df['TMAX'] = df['TMAX'].interpolate(method='linear')
        df['TMIN'] = df['TMIN'].interpolate(method='linear')
        df['PRCP'] = df['PRCP'].interpolate(method='linear')
        df['SNOW'] = df['SNOW'].interpolate(method='linear')
        df['SNWD'] = df['SNWD'].interpolate(method='linear')
        df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2

        for i in range(1, 6):
            df[f'TMAX_lag{i}'] = df['TMAX'].shift(i)
            df[f'TMIN_lag{i}'] = df['TMIN'].shift(i)
            df[f'TAVG_lag{i}'] = df['TAVG'].shift(i)
        df.dropna(inplace=True)
        lag_features = [f'TMAX_lag{i}' for i in range(1, 6)] + \
                       [f'TMIN_lag{i}' for i in range(1, 6)] + \
                       [f'TAVG_lag{i}' for i in range(1, 6)] + ['PRCP', 'SNOW', 'SNWD']

        X = df[lag_features]
        y_min = df['TMIN']
        y_max = df['TMAX']
        y_avg = df['TAVG']

        # Train-Test Split
        X_train, X_test, y_min_train, y_min_test, y_max_train, y_max_test, y_avg_train, y_avg_test = train_test_split(
            X, y_min, y_max, y_avg, test_size=0.2, random_state=42
        )

        model_min = RandomForestRegressor(n_estimators=30, max_depth=10,random_state=42)
        model_max = RandomForestRegressor(n_estimators=30, max_depth=10,random_state=42)
        model_avg = RandomForestRegressor(n_estimators=30, max_depth=10,random_state=42)

        model_min.fit(X_train, y_min_train)
        model_max.fit(X_train, y_max_train)
        model_avg.fit(X_train, y_avg_train)

        y_min_pred = model_min.predict(X_test)
        y_max_pred = model_max.predict(X_test)
        y_avg_pred = model_avg.predict(X_test)

        r2_min, adj_r2_min, mse_min, rmse_min, mae_min = evaluate_model(y_min_test, y_min_pred, X_test)
        r2_max, adj_r2_max, mse_max, rmse_max, mae_max = evaluate_model(y_max_test, y_max_pred, X_test)
        r2_avg, adj_r2_avg, mse_avg, rmse_avg, mae_avg = evaluate_model(y_avg_test, y_avg_pred, X_test)

        future_predictions = generate_future_predictions(df, model_min, model_max, model_avg, days=5, start_date='2024-11-19', lag_features=lag_features)

        output_text = f"City: {city_name}\n"
        output_text += f"TMIN Evaluation: R²: {r2_min:.4f}, Adjusted R²: {adj_r2_min:.4f}, MSE: {mse_min:.4f}, RMSE: {rmse_min:.4f}, MAE: {mae_min:.4f}\n"
        output_text += f"TMAX Evaluation: R²: {r2_max:.4f}, Adjusted R²: {adj_r2_max:.4f}, MSE: {mse_max:.4f}, RMSE: {rmse_max:.4f}, MAE: {mae_max:.4f}\n"
        output_text += f"TAVG Evaluation: R²: {r2_avg:.4f}, Adjusted R²: {adj_r2_avg:.4f}, MSE: {mse_avg:.4f}, RMSE: {rmse_avg:.4f}, MAE: {mae_avg:.4f}\n\n"
        output_text += "Future 5-Day Predictions:\n"
        output_text += future_predictions.to_string(index=False)
        output_text += "\n\n" + "-" * 50 + "\n\n"

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(output_text)

        print(f"Processed {city_name}, results appended to {output_file}")


Processed KBNA, results appended to output/AllCities_RandomForest.txt
Processed KBOI, results appended to output/AllCities_RandomForest.txt
Processed KDCA, results appended to output/AllCities_RandomForest.txt
Processed KDEN, results appended to output/AllCities_RandomForest.txt
Processed KDTW, results appended to output/AllCities_RandomForest.txt
Processed KIAH, results appended to output/AllCities_RandomForest.txt
Processed KJFK, results appended to output/AllCities_RandomForest.txt
Processed KMIA, results appended to output/AllCities_RandomForest.txt
Processed KMSP, results appended to output/AllCities_RandomForest.txt
Processed KOKC, results appended to output/AllCities_RandomForest.txt
Processed KORD, results appended to output/AllCities_RandomForest.txt
Processed KPDX, results appended to output/AllCities_RandomForest.txt
Processed KPHX, results appended to output/AllCities_RandomForest.txt
Processed KPWM, results appended to output/AllCities_RandomForest.txt
Processed KSAN, resu

4. XGBOOST: The characteristic of XGB is that the training speed is faster. The delayed value does not require additional filling to prevent strong fitting ability, but it is more sensitive to noise.

In [34]:
import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

def evaluate_model(y_test, y_pred, X_test):
    n = len(y_test)
    p = X_test.shape[1]
    r2 = r2_score(y_test, y_pred)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return r2, adj_r2, mse, rmse

def generate_future_predictions(data, model_min, model_max, model_avg, days=5, start_date=None, lag_features=[]):
    future_predictions = []
    current_data = data.iloc[-1][lag_features].values.reshape(1, -1)
    current_date = pd.to_datetime(start_date)

    for day in range(1, days + 1):
        pred_min = model_min.predict(current_data)[0]
        pred_max = model_max.predict(current_data)[0]
        pred_avg = model_avg.predict(current_data)[0]

        future_predictions.append({
            'DATE': current_date + pd.Timedelta(days=day),
            'TMIN': pred_min,
            'TMAX': pred_max,
            'TAVG': pred_avg
        })

        current_data = np.roll(current_data, -3)  
        current_data[0, :3] = [pred_max, pred_min, pred_avg]  # Insert new predictions

    return pd.DataFrame(future_predictions)

directory = '.'  
output_file = 'output/AllCities_XGBoost.txt' 
os.makedirs('output', exist_ok=True)  

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("XGBoost Model Results for All Cities:\n\n")
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        city_name = os.path.splitext(filename)[0]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
        df = df[df['DATE'] <= '2024-11-19']  # Filter data before or on 2024-11-19

        df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
        for i in range(1, 6):
            df[f'TMAX_lag{i}'] = df['TMAX'].shift(i)
            df[f'TMIN_lag{i}'] = df['TMIN'].shift(i)
            df[f'TAVG_lag{i}'] = df['TAVG'].shift(i)

        df.dropna(subset=['TMAX', 'TMIN', 'TAVG'], inplace=True)

        lag_features = [f'TMAX_lag{i}' for i in range(1, 6)] + \
                       [f'TMIN_lag{i}' for i in range(1, 6)] + \
                       [f'TAVG_lag{i}' for i in range(1, 6)] + ['PRCP', 'SNOW', 'SNWD']

        X = df[lag_features]
        y_min = df['TMIN']
        y_max = df['TMAX']
        y_avg = df['TAVG']

        # Train-Test Split
        X_train, X_test, y_min_train, y_min_test, y_max_train, y_max_test, y_avg_train, y_avg_test = train_test_split(
            X, y_min, y_max, y_avg, test_size=0.2, random_state=42
        )
        model_min = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6,
                                 subsample=0.8, colsample_bytree=0.8, random_state=42)
        model_max = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6,
                                 subsample=0.8, colsample_bytree=0.8, random_state=42)
        model_avg = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6,
                                 subsample=0.8, colsample_bytree=0.8, random_state=42)

        model_min.fit(X_train, y_min_train)
        model_max.fit(X_train, y_max_train)
        model_avg.fit(X_train, y_avg_train)

        y_min_pred = model_min.predict(X_test)
        y_max_pred = model_max.predict(X_test)
        y_avg_pred = model_avg.predict(X_test)

        r2_min, adj_r2_min, mse_min, rmse_min = evaluate_model(y_min_test, y_min_pred, X_test)
        r2_max, adj_r2_max, mse_max, rmse_max = evaluate_model(y_max_test, y_max_pred, X_test)
        r2_avg, adj_r2_avg, mse_avg, rmse_avg = evaluate_model(y_avg_test, y_avg_pred, X_test)

        future_predictions = generate_future_predictions(df, model_min, model_max, model_avg, days=5, start_date='2024-11-19', lag_features=lag_features)

        output_text = f"City: {city_name}\n"
        output_text += f"TMIN Evaluation: R²: {r2_min:.4f}, Adjusted R²: {adj_r2_min:.4f}, MSE: {mse_min:.4f}, RMSE: {rmse_min:.4f}\n"
        output_text += f"TMAX Evaluation: R²: {r2_max:.4f}, Adjusted R²: {adj_r2_max:.4f}, MSE: {mse_max:.4f}, RMSE: {rmse_max:.4f}\n"
        output_text += f"TAVG Evaluation: R²: {r2_avg:.4f}, Adjusted R²: {adj_r2_avg:.4f}, MSE: {mse_avg:.4f}, RMSE: {rmse_avg:.4f}\n\n"
        output_text += "Future 5-Day Predictions:\n"
        output_text += future_predictions.to_string(index=False)
        output_text += "\n\n" + "-" * 50 + "\n\n"

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(output_text)

        print(f"Processed {city_name}, results appended to {output_file}")


Processed KBNA, results appended to output/AllCities_XGBoost.txt
Processed KBOI, results appended to output/AllCities_XGBoost.txt
Processed KDCA, results appended to output/AllCities_XGBoost.txt
Processed KDEN, results appended to output/AllCities_XGBoost.txt
Processed KDTW, results appended to output/AllCities_XGBoost.txt
Processed KIAH, results appended to output/AllCities_XGBoost.txt
Processed KJFK, results appended to output/AllCities_XGBoost.txt
Processed KMIA, results appended to output/AllCities_XGBoost.txt
Processed KMSP, results appended to output/AllCities_XGBoost.txt
Processed KOKC, results appended to output/AllCities_XGBoost.txt
Processed KORD, results appended to output/AllCities_XGBoost.txt
Processed KPDX, results appended to output/AllCities_XGBoost.txt
Processed KPHX, results appended to output/AllCities_XGBoost.txt
Processed KPWM, results appended to output/AllCities_XGBoost.txt
Processed KSAN, results appended to output/AllCities_XGBoost.txt
Processed KSEA, results a

5.Arima: The model models the time sequence modeling by capturing the self -regression (AR), differential (i), and mobile average (MA) components of data.
Arima model model parameter selection: Use Auto_arima to automatically select parameters.
Let's choose the right parameter first, through Auto_arima, we get Tmax, Order = (5, 1, 3), TMIN, Order = (5, 1, 1), tavg, order = (5, 1, 3), to ensure the guarantee The reliability of running time, we choose to reduce the training time window, and select the data of the first 600 days as a data set. At the same time, the model evaluation is reduced, and only MSE and MAE are used.

In [37]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

def arima_forecast(series, order=(1, 1, 1), steps=5, start_date=None):
    model = ARIMA(series, order=order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=steps)
    future_dates = [start_date + pd.Timedelta(days=i) for i in range(1, steps + 1)]
    forecast_df = pd.DataFrame({'DATE': future_dates, 'FORECAST': forecast})
    return forecast_df

def evaluate_predictions(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    return mse, mae

directory = '.'  
output_file = 'output/AllCities_ARIMA.txt' 
os.makedirs('output', exist_ok=True)  

def combine_future_predictions(tmin_forecast_df, tmax_forecast_df, tavg_forecast_df):
    """Combine TMIN, TMAX, and TAVG forecasts into a single DataFrame."""
    combined_forecast = pd.DataFrame({
        'DATE': tmin_forecast_df['DATE'],
        'TMIN': tmin_forecast_df['FORECAST'],
        'TMAX': tmax_forecast_df['FORECAST'],
        'TAVG': tavg_forecast_df['FORECAST']
    })
    return combined_forecast

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        city_name = os.path.splitext(filename)[0]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)

        df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
        df = df[df['DATE'] <= '2024-11-19']  # Filter data before or on 2024-11-19

        df['TMAX'] = df['TMAX'].replace(0, np.nan)
        df['TMIN'] = df['TMIN'].replace(0, np.nan)
        df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2  
        df['TMAX'] = df['TMAX'].interpolate(method='linear')
        df['TMIN'] = df['TMIN'].interpolate(method='linear')
        df['TAVG'] = df['TAVG'].interpolate(method='linear')

        # Train-Test Split
        train_tmax = df['TMAX'][-600:-5]
        test_tmax = df['TMAX'][-5:]
        train_tmin = df['TMIN'][-600:-5]
        test_tmin = df['TMIN'][-5:]
        train_tavg = df['TAVG'][-600:-5]
        test_tavg = df['TAVG'][-5:]

        start_date = df['DATE'].max()
        tmax_arima_forecast_df = arima_forecast(train_tmax, order=(5, 1, 3), steps=5, start_date=start_date)
        tmin_arima_forecast_df = arima_forecast(train_tmin, order=(5, 1, 1), steps=5, start_date=start_date)
        tavg_arima_forecast_df = arima_forecast(train_tavg, order=(5, 1, 3), steps=5, start_date=start_date)

        combined_forecast_df = combine_future_predictions(tmin_arima_forecast_df, tmax_arima_forecast_df, tavg_arima_forecast_df)

        # Evaluate ARIMA predictions
        tmax_arima_mse, tmax_arima_mae = evaluate_predictions(test_tmax, tmax_arima_forecast_df['FORECAST'])
        tmin_arima_mse, tmin_arima_mae = evaluate_predictions(test_tmin, tmin_arima_forecast_df['FORECAST'])
        tavg_arima_mse, tavg_arima_mae = evaluate_predictions(test_tavg, tavg_arima_forecast_df['FORECAST'])

        output_text = f"City: {city_name}\n"
        output_text += f"TMAX - MSE: {tmax_arima_mse:.2f}, MAE: {tmax_arima_mae:.2f}\n"
        output_text += f"TMIN - MSE: {tmin_arima_mse:.2f}, MAE: {tmin_arima_mae:.2f}\n"
        output_text += f"TAVG - MSE: {tavg_arima_mse:.2f}, MAE: {tavg_arima_mae:.2f}\n\n"
        output_text += "Future 5-Day Predictions (Combined):\n"
        output_text += combined_forecast_df.to_string(index=False)
        output_text += "\n\n" + "-" * 50 + "\n\n"
        
        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(output_text)

        print(f"Processed {city_name}, results appended to {output_file}")



Processed KBNA, results appended to output/AllCities_ARIMA.txt
Processed KBOI, results appended to output/AllCities_ARIMA.txt
Processed KDCA, results appended to output/AllCities_ARIMA.txt
Processed KDEN, results appended to output/AllCities_ARIMA.txt
Processed KDTW, results appended to output/AllCities_ARIMA.txt
Processed KIAH, results appended to output/AllCities_ARIMA.txt
Processed KJFK, results appended to output/AllCities_ARIMA.txt
Processed KMIA, results appended to output/AllCities_ARIMA.txt
Processed KMSP, results appended to output/AllCities_ARIMA.txt
Processed KOKC, results appended to output/AllCities_ARIMA.txt
Processed KORD, results appended to output/AllCities_ARIMA.txt
Processed KPDX, results appended to output/AllCities_ARIMA.txt
Processed KPHX, results appended to output/AllCities_ARIMA.txt
Processed KPWM, results appended to output/AllCities_ARIMA.txt
Processed KSAN, results appended to output/AllCities_ARIMA.txt
Processed KSEA, results appended to output/AllCities_AR